In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn

import healpy as hp
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db


In [2]:
plt.rc('legend', fontsize=20) # using a size in points
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rcParams['axes.labelsize'] = 16

In [3]:
# load opsim database
dbpath_v15 = "/home/idies/workspace/lsst_cadence/FBS_1.5/"  # path to all opsim databases

dbpath_v17 = "/home/idies/workspace/lsst_cadence/FBS_1.7/"

dbpath_v171 = "/home/idies/workspace/lsst_cadence/FBS_1.7.1/"


# output directory
dataRawDir = '/home/idies/workspace/Temporary/lixl/scratch/outDir/tGaps/'

outDir = '/home/idies/workspace/Temporary/lixl/scratch/outDir/resultsDb/'
resultsDb = db.ResultsDb(outDir=outDir)


In [4]:
# get the name of all opsim dbs 
import glob

workpath = os.getcwd()
#workpath = '/home/idies/workspace/Storage/lixl/persistent/LSST_OpSim/unknowns'

os.chdir(dbpath_v15)  # change to opsim database directory
dblist_v15 = glob.glob('*.db') 

os.chdir(dbpath_v17)  # change to opsim database directory
dblist_v17 = glob.glob('*.db') 

os.chdir(dbpath_v171)  # change to opsim database directory
dblist_v171 = glob.glob('*.db') 

os.chdir(workpath) # change back to work directory

dblist_v15.sort()
dblist_v17.sort()
dblist_v171.sort()



In [32]:
class depthMetric(metrics.BaseMetric):
    """
    returns
        median of fiveSigmaDepth 
    Parameters:
        colname: 
        fltpair: filter eg 'r'
    
    """

    def __init__(self, colname=['observationStartMJD', 'filter', 'fiveSigmaDepth'], flt='r',
                 dataout=True, **kwargs):
        self.colname = colname
        self.flt = flt
        self.dataout = dataout
        self.Nrun = 0   # record the how many time run run()
        
        if self.dataout:
            super().__init__(col=self.colname, metricDtype='object', **kwargs)
        else:
            super().__init__(col=self.colname, metricDtype='float', **kwargs)
            
    def RADec2pix(self, nside, ra, dec, degree=True):
        """
        Calculate the nearest healpixel ID of an RA/Dec array, assuming nside.
    
        Parameters
        ----------
        nside : int
            The nside value of the healpix grid.
        ra : numpy.ndarray
            The RA values to be converted to healpix ids, in degree by default.
        dec : numpy.ndarray
            The Dec values to be converted to healpix ids, in degree by default.
    
        Returns
        -------
        numpy.ndarray
            The healpix ids.
        """
        if degree:
            ra = np.radians(ra) # change to radians
            dec = np.radians(dec)
        
        lat = np.pi/2. - dec
        hpid = hp.ang2pix(nside, lat, ra )
        return hpid


    def run(self, dataSlice, slicePoint=None):
        
        # return all possible time gaps for each fields
        
        #check input config
        #print(f0, f1, self.tmin, self.tmax, self.mag_lim)
            
        # sort dataSlice
        idx =  dataSlice['filter'] == self.flt 
        
        dataSlice['fiveSigmaDepth'][idx]
        
        fieldRA = np.mean(dataSlice['fieldRA']),
        fieldDec = np.mean(dataSlice['fieldDec']),
        
        fiveSigmaMedian = np.median( dataSlice['fiveSigmaDepth'][idx] )
        
        dic = {'Nrun': self.Nrun, 
                'pixId': self.RADec2pix(nside=16, ra=fieldRA, dec=fieldDec)[0],
                #'Nv': Nv,
                #'Dkl': Dkl,
                # 'FoM_i': FoM_i,
                'fiveSigmaMedian': np.median( fiveSigmaMedian )
                  }
        self.Nrun += 1
        
        if self.dataout:
            # return dT
            result = dic
            return result
        else:
            #f0 = self.fltpair[0]
            #f1 = self.fltpair[1]
            #result = np.min(dT) if len(dT)!=0 else np.inf
            result = fiveSigmaMedian
            return float(result) 


In [18]:
def pix2radec(nside, ipix, degree=True):
    """from ipix to ra/dec"""
    
    theta, phi = hp.pix2ang(nside, ipix)
    ra = phi
    dec = np.pi/2. - theta
    if degree:
        ra = np.rad2deg(ra)
        dec = np.rad2deg(dec)
    return ra, dec

def runMetric(dbname, dbpath, flt, ipix_ms=None, nside=16, sqlstr=None,
              colname=['observationStartMJD', 'filter', 'fiveSigmaDepth'],
               dataout=True, **kwargs
             ):
    """run filterPairTGapsMetric
    dbname: name of opsim database
    fltpair: filter pair, eg: 'ri'
    ipix_ms: pixIds of footprint
    """
    opsdb = db.OpsimDatabase(dbpath+dbname)
    
    #fltpair = ['r', 'i']
    
    if ipix_ms!=None:
        # use UserPointsSlicer for minisurvey
        ra, dec = pix2radec(nside=nside, ipix=ipix_ms)
        slicer = slicers.UserPointsSlicer(ra, dec, latLonDeg=True, badval=0)
    else:
        # use HealpixSlicer 
        slicer = slicers.HealpixSlicer(nside=nside)
    
    # metric, slicer, constraint
    metric = depthMetric( colname=colname, flt=flt, 
                  dataout=dataout, **kwargs)
    
    sqlconstraint = 'filter in ("{}")'.format(flt)
    if sqlstr!=None:
        sqlconstraint += ' and {}'.format(sqlstr)
        
    # bundle
    newMetricSky = metricBundles.MetricBundle(metric, slicer, sqlconstraint)
    
    # group bundle
    bundleDict = {'newMetricSky':newMetricSky}
    group = metricBundles.MetricBundleGroup(bundleDict, opsdb, outDir=outDir, resultsDb=resultsDb)
    
    # run
    group.runAll()
    #group.plotAll(closefigs = False)
    return newMetricSky



### run for one opsim one filter

In [11]:
dblist_v15[5]


'baseline_v1.5_10yrs.db'

In [36]:
flt = 'r'
dbname = dblist_v15[5]
dbpath = dbpath_v15

metricSky = runMetric(dbname, dbpath, flt, ipix_ms=None, nside=16, sqlstr=None,
              colname=['observationStartMJD', 'filter', 'fiveSigmaDepth'])


Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("r") for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 491676 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [34]:
arr = metricSky.metricValues.data
arr_valid = arr[ arr != None ]
df = pd.DataFrame.from_records(arr_valid)
df.head(3)

,Nrun,fiveSigmaMedian,pixId
0,0,23.319188,621
1,1,22.854614,622
2,2,23.452739,623


### run for all filters all opsims

In [37]:
filters = ['u', 'g', 'r', 'i', 'z', 'y']

In [47]:
%%time
# run all filters for all opsims 

nside = 16
msname = 'WFD'

if 'LMC' in msname:
    print(f"running {msname}")
    ipix_ms = ipix_LMC
elif 'SMC' in msname:
    print(f"running {msname}")
    ipix_ms = ipix_SMC
elif 'GP' in msname:
    print(f"running {msname}")
    ipix_ms = ipix_GP
else:
    ipix_ms = None

#sqlstr = "night<4000 and proposalId=1"
#sqlstr = None
if 'WFD' in msname:
    sqlstr = "proposalId=1"
else:
    sqlstr = "note not like '%DD%'"

#--------------------------------
# empty metricArr 
#diclist = []
#for Nrun in range( len(ipix_ms) ):
#    dic = {#'Nrun': Nrun, 
#                'pixId': ipix_ms[Nrun],
#                'fiveSigmaMedian': 0
#                  }
#    diclist.append(dic)
#arr_empty = np.array(diclist)
#df_empty = pd.DataFrame.from_records( arr_empty )
#--------------------------------

dbpath = dbpath_v15

df_depth = pd.DataFrame(columns=['db', 'u', 'g', 'r', 'i', 'z', 'y'])

depthRow = {}
for dbname in dblist_v15:
    #dataRaw = {}
    # dataRaw[dbname] = {}
    depthRow['db'] = dbname
    for flt in filters: 

        # dataRaw[dbname][fltpair] = {}
        metricSky = runMetric(dbname, dbpath, flt, ipix_ms=None, nside=16, sqlstr=sqlstr,
              colname=['observationStartMJD', 'filter', 'fiveSigmaDepth'])

        arr = metricSky.metricValues.data 
                
        df = pd.DataFrame.from_records( arr[ arr != None ] )
        
        print(">>> ", dbname, flt, df.shape)

        if len(df)==0:
            df = df_empty
        
        depthRow[flt] = df['fiveSigmaMedian'].median()
        
        # dataRaw[dbname][fltpair] = df[['pixId', 'fiveSigmaMedian', ]] 
        
        # save raw data to pickle
        # pd.DataFrame( dataRaw ).to_pickle(dataRawDir+'/tgaps_{}_{}.pkl'.format(dbname, msname))
    df_depth = df_depth.append(depthRow, ignore_index=True)

    df_depth.to_csv(f"data/df_depth_v1.5_{msname}", index=False)


Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("u") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 119740 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  agnddf_v1.5_10yrs.db u (1458, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("g") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 171561 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  agnddf_v1.5_10yrs.db g (1459, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("r") and proposal

Found 158638 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  baseline_2snaps_v1.5_10yrs.db g (1460, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("r") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 373468 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  baseline_2snaps_v1.5_10yrs.db r (1462, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("i") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 375626 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Complete

Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  bulges_bs_v1.5_10yrs.db r (1460, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("i") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 338488 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  bulges_bs_v1.5_10yrs.db i (1462, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("z") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 302878 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  bulges_bs_v1.5_10yrs.db z (1462, 3)
Healpix slicer 

Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  bulges_cadence_bulge_wfd_v1.5_10yrs.db i (1460, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("z") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 299997 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  bulges_cadence_bulge_wfd_v1.5_10yrs.db z (1462, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("y") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 318453 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  bulges_cadence_bulge_

Found 346758 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  daily_ddf_v1.5_10yrs.db z (1460, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("y") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 363588 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  daily_ddf_v1.5_10yrs.db y (1461, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("u") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 120292 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  dcr_

...

Found 149570 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  wfd_depth_scale0.65_noddf_v1.5_10yrs.db g (1461, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("r") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 344185 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  wfd_depth_scale0.65_noddf_v1.5_10yrs.db r (1461, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("i") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 346731 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary 

Found 340938 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  wfd_depth_scale0.70_v1.5_10yrs.db r (1462, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("i") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 343652 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  wfd_depth_scale0.70_v1.5_10yrs.db i (1461, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("z") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 308633 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.


Found 393671 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  wfd_depth_scale0.80_noddf_v1.5_10yrs.db i (1461, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("z") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 352820 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  wfd_depth_scale0.80_noddf_v1.5_10yrs.db z (1460, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("y") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 370860 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary 

Found 351227 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  wfd_depth_scale0.85_v1.5_10yrs.db z (1460, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("y") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 369222 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  wfd_depth_scale0.85_v1.5_10yrs.db y (1460, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("u") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 124143 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.


Found 418114 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  wfd_depth_scale0.95_noddf_v1.5_10yrs.db y (1461, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("u") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 119668 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
>>>  wfd_depth_scale0.95_v1.5_10yrs.db u (1459, 3)
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint filter in ("g") and proposalId=1 for columns ['fieldDec', 'filter', 'observationStartMJD', 'fieldRA', 'fiveSigmaDepth']
Found 178572 visits
Running:  ['newMetricSky']
Completed metric generation.
Running reduce methods.
Running summary statis

In [38]:
ipix_LMC = [2899, 2900, 2901, 2934, 2935, 2936, 2965, 2966, 2967, 2992, 2993, 2994]
ipix_SMC = [2960, 2961, 2988, 2989, 3012]
ipix_GP = [ 144,  145,  146,  147,  179,  180,  181,  182,  183,  184,  217,
        218,  219,  225,  226,  260,  261,  271,  272,  307,  308,  320,
        321,  358,  374,  413,  431,  432,  471,  472,  493,  534,  535,
        558,  598,  599,  622,  661,  662,  687,  725,  726,  751,  788,
        789,  790,  815,  816,  852,  853,  854,  879,  915,  916,  917,
        944,  979,  980,  981, 1008, 1042, 1043, 1044, 1045, 1073, 1107,
       1108, 1109, 1136, 1170, 1171, 1172, 1201, 1234, 1235, 1236, 1237,
       1265, 1297, 1298, 1299, 1300, 1330, 1361, 1362, 1363, 1364, 1393,
       1425, 1426, 1427, 1428, 1458, 1489, 1490, 1491, 1492, 1522, 1552,
       1553, 1554, 1555, 1587, 1616, 1617, 1618, 1619, 1620, 1650, 1679,
       1680, 1681, 1682, 1683, 1715, 1744, 1745, 1746, 1747, 1779, 1807,
       1808, 1809, 1810, 1811, 1843, 1871, 1872, 1873, 1874, 1875, 1907,
       1934, 1935, 1936, 1937, 1938, 1972, 1998, 1999, 2000, 2001, 2002,
       2003, 2036, 2062, 2063, 2064, 2065, 2066, 2100, 2126, 2127, 2128,
       2129, 2130, 2164, 2189, 2190, 2191, 2192, 2193, 2194, 2229, 2253,
       2254, 2255, 2256, 2257, 2258, 2293, 2316, 2317, 2318, 2319, 2320,
       2321, 2358, 2380, 2381, 2382, 2383, 2384, 2385, 2421, 2422, 2443,
       2444, 2445, 2446, 2447, 2448, 2449, 2486, 2487, 2507, 2508, 2509,
       2510, 2511, 2512, 2513, 2550, 2551, 2570, 2571, 2572, 2573, 2574,
       2575, 2576, 2614, 2630, 2631, 2632, 2633, 2634, 2635, 2636, 2673,
       2687, 2688, 2689, 2690, 2691, 2692, 2693, 2728, 2729, 2739, 2740,
       2741, 2742, 2743, 2744, 2745, 2746, 2779, 2780, 2781, 2787, 2788,
       2789, 2790, 2791, 2792, 2793, 2794, 2826, 2827, 2828, 2829, 2830,
       2831, 2832, 2833, 2834, 2835, 2836, 2837, 2838, 2839, 2869, 2870,
       2871, 2872, 2873, 2874, 2875, 2876, 2877, 2878, 2879, 2908, 2909,
       2910, 2911, 2912, 2913, 2914, 2915, 2916, 2943, 2944, 2945, 2946,
       2947, 2948, 2974, 2975, 2976]



#### plot depth distribution

In [ ]:
%%time
# run all filters for all opsims 

nside = 16
msname = 'WFD'

if 'LMC' in msname:
    print(f"running {msname}")
    ipix_ms = ipix_LMC
elif 'SMC' in msname:
    print(f"running {msname}")
    ipix_ms = ipix_SMC
elif 'GP' in msname:
    print(f"running {msname}")
    ipix_ms = ipix_GP
else:
    ipix_ms = None

#sqlstr = "night<4000 and proposalId=1"
#sqlstr = None
if 'WFD' in msname:
    sqlstr = "proposalId=1"
else:
    sqlstr = "note not like '%DD%'"

#--------------------------------
# empty metricArr 
#diclist = []
#for Nrun in range( len(ipix_ms) ):
#    dic = {#'Nrun': Nrun, 
#                'pixId': ipix_ms[Nrun],
#                'fiveSigmaMedian': 0
#                  }
#    diclist.append(dic)
#arr_empty = np.array(diclist)
#df_empty = pd.DataFrame.from_records( arr_empty )
#--------------------------------

dbpath = dbpath_v15

df_depth = pd.DataFrame(columns=['db', 'u', 'g', 'r', 'i', 'z', 'y'])

depthRow = {}
for dbname in dblist_v15:
    #dataRaw = {}
    # dataRaw[dbname] = {}
    depthRow['db'] = dbname
    for flt in filters: 

        # dataRaw[dbname][fltpair] = {}
        metricSky = runMetric(dbname, dbpath, flt, ipix_ms=None, nside=16, sqlstr=sqlstr,
              colname=['observationStartMJD', 'filter', 'fiveSigmaDepth'])

        arr = metricSky.metricValues.data 
                
        df = pd.DataFrame.from_records( arr[ arr != None ] )
        
        print(">>> ", dbname, flt, df.shape)

        if len(df)==0:
            df = df_empty
        
        depthRow[flt] = df['fiveSigmaMedian'].median()
        
        # dataRaw[dbname][fltpair] = df[['pixId', 'fiveSigmaMedian', ]] 
        
        # save raw data to pickle
        # pd.DataFrame( dataRaw ).to_pickle(dataRawDir+'/tgaps_{}_{}.pkl'.format(dbname, msname))
    df_depth = df_depth.append(depthRow, ignore_index=True)

    df_depth.to_csv(f"data/df_depth_v1.5_{msname}", index=False)

